In [1]:
import sys
sys.path.extend(['..', '../', '../..', '../../' ])
%load_ext autoreload
%autoreload 2

# LeNet5 - Cat-Vs-Dog

In [2]:
import paths
import keras
from src.parameters import metrics
from src.datasets.dataset_generators import DatasetGenerators
from src.keras_callback.time_history import TimeHistory
from src.export.csv_export import export_learning_history_to_csv, export_learning_validation_result_to_csv
from src.export.model_export import clear_exported_models
from src.keras_model.lenet5_network import LeNet5Network

Using TensorFlow backend.


In [3]:
network_name = 'lenet5'
fit_epochs = 50
dataset = 'cat-vs-dog'
dataset_percentage = '10'

In [4]:
network = LeNet5Network()

In [5]:
dataset_generators = DatasetGenerators(
    name=f'{dataset}_{dataset_percentage}',
    dataset_dir=paths.data_location,
    image_size=network.image_size,
    batch_size=network.batch_size,
    grayscale=network.grayscale
)

Loading train set for cat-vs-dog_10...
Found 1750 images belonging to 2 classes.
Loading test set for cat-vs-dog_10...
Found 500 images belonging to 2 classes.
Loading validation set for cat-vs-dog_10...
Found 250 images belonging to 2 classes.


In [6]:
model = network.get_compiled_model(
    classes=dataset_generators.test.num_classes
)

In [7]:
checkpoint_path = f'{paths.models_location}{network_name}/{dataset}/'
best_model_path = f'{checkpoint_path}best.h5'
clear_exported_models(checkpoint_path)

checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=f'{checkpoint_path}{{epoch:02d}}-{{val_accuracy}}.h5',
    save_best_only=False,
    save_weights_only=False
)

best_callback = keras.callbacks.ModelCheckpoint(
    filepath=best_model_path,
    save_best_only=True,
    verbose=1,
    save_weights_only=False
)

time_callback = TimeHistory()

model_callbacks = [checkpoint_callback, best_callback, time_callback]

In [8]:
fit_result = model.fit_generator(
    generator=dataset_generators.train,
    epochs=fit_epochs,
    validation_data=dataset_generators.test,
    callbacks = model_callbacks
)

Epoch 1/50
55/55 [==============================] - 5s 98ms/step - loss: 2.9391 - accuracy: 0.5366 - precision: 0.4966 - recall: 0.5371 - val_loss: 0.8209 - val_accuracy: 0.5380 - val_precision: 0.5445 - val_recall: 0.4961

Epoch 00001: val_loss improved from inf to 0.82088, saving model to /Users/igor/Python/CNN-Architecture-Comparison/models/lenet5/cat-vs-dog/best.h5
Epoch 2/50
55/55 [==============================] - 6s 108ms/step - loss: 0.6976 - accuracy: 0.6086 - precision: 0.6132 - recall: 0.6099 - val_loss: 0.9688 - val_accuracy: 0.5700 - val_precision: 0.5757 - val_recall: 0.5935

Epoch 00002: val_loss did not improve from 0.82088
Epoch 3/50
55/55 [==============================] - 7s 119ms/step - loss: 0.5911 - accuracy: 0.6880 - precision: 0.6864 - recall: 0.7055 - val_loss: 0.7835 - val_accuracy: 0.5920 - val_precision: 0.5846 - val_recall: 0.6286

Epoch 00003: val_loss improved from 0.82088 to 0.78355, saving model to /Users/igor/Python/CNN-Architecture-Comparison/models/l

In [9]:
best_model = keras.models.load_model(best_model_path,
    custom_objects= {
        'precision': metrics.precision,
        'recall': metrics.recall
    }
)
validation_result = best_model.evaluate_generator(
    generator=dataset_generators.validate
)

## Results

### Epoch CSV

In [10]:
csv_file_path = f'{paths.results_location}{network_name}/{dataset}-history.csv'
export_learning_history_to_csv(csv_file_path, fit_result, time_callback.times)

epoch, time, val_loss, val_accuracy, val_precision, val_recall
0,0,0,0,0,0
1,5.437490224838257,0.820881724357605,0.5379999876022339,0.5444971919059753,0.496118426322937
2,5.9510321617126465,0.968769371509552,0.5699999928474426,0.5756964683532715,0.5935468673706055
3,6.619992971420288,0.7835481762886047,0.5920000076293945,0.5846453905105591,0.6286236643791199
4,7.46208381652832,0.6172042489051819,0.5740000009536743,0.5835943222045898,0.5173345804214478
5,7.748788118362427,0.7130295634269714,0.6100000143051147,0.5767063498497009,0.8349744081497192
6,7.293951034545898,0.7503093481063843,0.621999979019165,0.6453566551208496,0.6087559461593628
7,8.052918195724487,0.8311079144477844,0.5960000157356262,0.6546306014060974,0.43048423528671265
8,7.448494911193848,0.8738366961479187,0.6480000019073486,0.6242109537124634,0.6999671459197998
9,5.733583688735962,0.9307301640510559,0.6439999938011169,0.6080463528633118,0.7482956051826477
10,6.16067910194397,1.2295809984207153,0.6200000047683716,0.5893

### Validation result

In [11]:
csv_file_path = f'{paths.results_location}{network_name}/{dataset}-result.csv'
export_learning_validation_result_to_csv(csv_file_path, validation_result, time_callback.times)

avg_epoch_time, time, val_loss, val_accuracy, val_precision, val_recall
4.561804723739624,228.0902361869812,0.7793372869491577,0.6119999885559082,0.624394953250885,0.5814825892448425
